### module import protection

In [ ]:
if globals().get('LOADED_DATA_CONTAINER') == None:
    display('LOADED_DATA_CONTAINER')
    LOADED_DATA_CONTAINER=True

### modules

In [ ]:
if globals().get('LOADED_ANNOTATIONS') == None:
    %run ANNOTATIONS.ipynb

In [ ]:
if globals().get('LOADED_PATTERN_OBSERVER') == None:
    %run PATTERN_OBSERVER.ipynb

### imports

In [ ]:
from collections import namedtuple
from pathlib import Path
from functools import reduce
import pandas as pd

### begin

In [ ]:
'''
SUBJECT
- VIEWING_TABS observes DATA_CONTAINER to populate itself

OBSERVER
- DATA_CONTAINER observes SETTINGS for repopulation purposes
- DATA_CONTAINER observes BACKTEST_SELECTOR for population purposes
- DATA_CONTAINER observes BOOK_SELECTOR for population purposes
'''
class DATA_CONTAINER(IObserver,ISubject):
    ####################################
    # static
    ####################################
    # standardized file names
    BTPATHTUPLE      = namedtuple('BTPATHTUPLE',['backtest_name','subdir','filename'])
    _dev             = BTPATHTUPLE('./','./',None)
    _daily           = BTPATHTUPLE(None,'./daily','summary_YYYYMMDD')
    _extraday        = BTPATHTUPLE(None,'./extraday','BOOK_YYYYMMDD')
    _intraday        = BTPATHTUPLE(None,'./intraday','BOOK_YYYYMMDD')
    _factor          = BTPATHTUPLE(None,'./factor','BOOK_YYYYMMDD')

    ####################################
    # constructor
    ####################################
    def __init__(self,
        name                  : str = 'DATA_CONTAINER',
        applied_backtest_path : str = r'C:\Users\ahkar\OneDrive\Documents\Data\B3',
        applied_view_type     : str = None,
        applied_plot_type     : str = None,
        applied_return_type   : str = None,
        applied_date_from           = None,
        applied_date_to             = None,
        selected_backtests          = None,
        selected_books              = None
        ):
        self.name                   = name
        self._applied_backtest_path = applied_backtest_path
        self._applied_view_type     = applied_view_type
        self._applied_plot_type     = applied_plot_type
        self._applied_return_type   = applied_return_type
        self._selected_backtests    = selected_backtests if selected_backtests != None else self._populate_selected_backtests()
        self._selected_books        = selected_books if selected_books != None else self._populate_selected_books()
        self._data_dico             = {}
                
        # for observer pattern
        self._observers             = []
        self.reference              = {
            'applied_view_type'     : applied_view_type,
            'applied_plot_type'     : applied_plot_type,
            'applied_return_type'   : applied_return_type,
            'applied_date_from'     : applied_date_from,
            'applied_date_to'       : applied_date_to,
            'applied_backtest_path' : applied_backtest_path,
            'df'                    : pd.DataFrame(),
        }
            
        self._logging() # show internals
        
    ####################################
    # populate defaults
    ####################################
    def _populate_selected_backtests(self) -> OPTIONID_VALUES :
        return {
            ('AZUL4.SA',Path(r'C:/Users/ahkar/OneDrive/Documents/Data/B3/AZUL4.SA.csv')) : True,
            ('EMBR3.SA',Path(r'C:/Users/ahkar/OneDrive/Documents/Data/B3/EMBR3.SA.csv')) : True,
            ('ECOR3.SA',Path(r'C:/Users/ahkar/OneDrive/Documents/Data/B3/ECOR3.SA.csv')) : False,
        }

    def _populate_selected_books(self) -> OPTIONID_VALUES :
        return {
            ('Trading','Trading') : True,
            ('Quote','Quote')     : True,
            ('MM2','MM2')         : True,
            ('Hedge','Hedge')     : True,
            ('Hit','Hit')         : False,
        }

    ####################################
    # observer pattern
    ####################################
    # subject
    def attach(self,observer : IObserver) -> None :
        print('OBSERVER PATTERN',':',observer.name,'OBSERVES',self.name)
        self._observers.append(observer)
        
    def detach(self,observer : IObserver) -> None :
        print('OBSERVER PATTERN',':',observer.name,'STOPS OBSERVING',self.name)
        self._observers.remove(observer)
        
    def notify(self,info) -> None :
        print('OBSERVER PATTERN',':',self.name,'NOTIFIES',len(self._observers),'OBSERVERS')
        
        # print receivers
        for observer in self._observers:
            print('OBSERVER PATTERN',':',self.name,'NOTIFIES',observer.name)
            observer.react(self.name,info)

    # observer
    def react(self,
        subject_name : str,
        subject_info : object
        ) -> None :
        print('OBSERVER PATTERN',':',self.name,'REACTS','subject_name',subject_name)
        print('OBSERVER PATTERN',':',self.name,'REACTS','subject_info',subject_info.keys())

        # SETTINGS changed
        if subject_name == 'SETTINGS':
            if 'reload_data' in subject_info:
                # told by SETTINGS to reload all backtests
                print(self.name,':','reload_data')
                
                self._data_dico.clear() # empty data_dico and reload
                self._logging() # show internals

                self.reference['df']                    = self._build_df(self.reference['applied_view_type']) # updates self._data_dico as well
            else:
                print(self.name,':','update settings')
                
                '''
                this block is probably redundant
                # empty data_dico if SETTINGS filepath OR view_type changed
                if (
                    self.reference['applied_backtest_path'] != Path(subject_info['backtest_path']) or # path changed
                    self.reference['applied_view_type']     != subject_info['view_type']              # view type changed
                ):
                    print('OBSERVER PATTERN',':',self.name,'clear dico')
                    self._data_dico.clear() # TODO , dont clear dico, ensure self._selected_backtests is emptied instead
                this block is probably redundant
                '''
                
                # update
                self.reference['applied_view_type']     = subject_info['view_type']
                self.reference['applied_plot_type']     = subject_info['plot_type']
                self.reference['applied_return_type']   = subject_info['return_type']
                self.reference['applied_date_from']     = subject_info['date_from']
                self.reference['applied_date_to']       = subject_info['date_to']
                self.reference['applied_backtest_path'] = subject_info['backtest_path']
                self.reference['df']                    = self._build_df(self.reference['applied_view_type'])
        
        # BACKTEST_SELECTOR changed
        if subject_name == 'BACKTEST_SELECTOR':
            self._selected_backtests                    = subject_info
            self.reference['df']                        = self._build_df(self.reference['applied_view_type'])

        # BOOK_SELECTOR changed
        if subject_name == 'BOOK_SELECTOR':
            self._selected_books                        = subject_info

        # BACKTEST_VIEWER changed
        if subject_name == 'BACKTEST_VIEWER':
            print(self.name,'BACKTEST_VIEWER force replot via DATA_CONTAINER')
            
        # logging
        self._logging()
        
        # notify
        self.notify(self.reference)
    
    ####################################
    # functions
    ####################################
    def _extraday_set_index(self,df : pd.DataFrame) -> pd.DataFrame :
        df.timestamp = pd.to_datetime(df.timestamp)
        return df.set_index('timestamp')

    def _build_df(self,view_type : str) -> pd.DataFrame : # TODO, this needs to depend on `view_type`
        print(self.name,':','_build_df','view_type =',view_type)

        ##################
        # add / remove as per checks
        ##################
        for (option_name,option_path),checked in self._selected_backtests.items():
            if checked:
                self._data_dico_add_key(option_name,option_path) # make sure present
            else:
                self._data_dico_remove_key(option_name,option_path) # make sure not present

        ##################
        # remove options no longer in range from self._data_dico
        ##################
        remove=[]
        for (option_name,option_path) in self._data_dico.keys():
            if not (option_name,option_path) in self._selected_backtests.keys():
                remove.append((option_name,option_path))
        
        for (option_name,option_path) in remove:
            self._data_dico_remove_key(option_name,option_path)
        
        ##################
        # bail if empty dico
        ##################
        if not self._data_dico:
            return pd.DataFrame()

        ##################
        # build df (dico not empty)
        ##################
        # flatten
        df = pd.concat(
            map(self._extraday_set_index,self._data_dico.values()),
            keys=[option_name for (option_name,option_path) in self._data_dico.keys()],
            names=['sym']
        )

        # reorder
        df = df.reorder_levels([1,0])
        
        ##################
        # apply filters
        ##################
        if self.reference['applied_view_type'] == 'Extraday':
            # apply lower date filter
            if not self.reference['applied_date_from'] is None:
                df = df[df.index.get_level_values('timestamp').date >= self.reference['applied_date_from']]
            # apply upper date filter
            if not self.reference['applied_date_to'] is None:
                df = df[df.index.get_level_values('timestamp').date <= self.reference['applied_date_to']]
        
        if self.reference['applied_view_type'] == 'Intraday':
            # apply lower time filter
            if not self.reference['applied_time_from'] is None:
                df = df[df.index.get_level_values('timestamp').time >= self.reference['applied_time_from']]
            # apply upper time filter
            if not self.reference['applied_time_to'] is None:
                df = df[df.index.get_level_values('timestamp').time <= self.reference['applied_time_to']]

        ##################
        # return
        ##################
        return df 

    '''
    d=DATA_CONTAINER()
    d._data_dico_add_key('BRFS3.SA',Path(r'C:/Users/ahkar/OneDrive/Documents/Data/B3/BRFS3.SA.csv'))
    d._data_dico_add_key('VALE3.SA',Path(r'C:/Users/ahkar/OneDrive/Documents/Data/B3/VALE3.SA.csv'))
    d._data_dico_add_key('VALE3.SA',Path(r'C:/Users/ahkar/OneDrive/Documents/Data/B3/VALE3.SA.csv'))
    d._data_dico_add_key('VALE3.SA',Path(r'C:/Users/ahkar/OneDrive/Documents/Data/B3/VALE3.SA.csv'))
    '''
    def _data_dico_add_key(self,
        option_name : str,
        option_path : Path,
        view_type   : str = 'dev',
        ) -> None :
        '''make sure data is populated'''
        if not (option_name,option_path) in self._data_dico:
            self._data_dico[(option_name,option_path)]=pd.read_csv(self._get_filepath(option_name,option_path,view_type))
            print(self.name,':','KEYS =',len(self._data_dico),':',(option_name,option_path),'ADD')
        else:
            print(self.name,':','KEYS =',len(self._data_dico),':',(option_name,option_path),'pass (no add)')
            pass
    
    def _data_dico_remove_key(self,
        option_name : str,
        option_path : Path,
        ) -> None :
        '''make sure data removed from data_dico'''
        if (option_name,option_path) in self._data_dico:
            del self._data_dico[(option_name,option_path)] # remove from dico
            print(self.name,':','KEYS =',len(self._data_dico),':',(option_name,option_path),'REMOVE')
        else:
            print(self.name,':','KEYS =',len(self._data_dico),':',(option_name,option_path),'pass (no remove)')
            pass

    '''
    d=DATA_CONTAINER()
    print(d._get_filepath('BRFS3.SA',Path(r'C:/Users/ahkar/OneDrive/Documents/Data/B3/BRFS3.SA.csv'),view_type='dev'))
    print(d._get_filepath('BRFS3.SA',Path(r'C:/Users/ahkar/OneDrive/Documents/Data/B3/BRFS3.SA.csv'),view_type='daily'))
    print(d._get_filepath('BRFS3.SA',Path(r'C:/Users/ahkar/OneDrive/Documents/Data/B3/BRFS3.SA.csv'),view_type='intraday'))
    print(d._get_filepath('BRFS3.SA',Path(r'C:/Users/ahkar/OneDrive/Documents/Data/B3/BRFS3.SA.csv'),view_type='extraday'))
    print(d._get_filepath('BRFS3.SA',Path(r'C:/Users/ahkar/OneDrive/Documents/Data/B3/BRFS3.SA.csv'),view_type='factor'))
    '''
    def _get_filepath(self,
        option_name : str,
        option_path : Path,
        view_type   : str = 'dev',
        ) -> Path :
        # build
        BackestPathTuple = eval('DATA_CONTAINER.'+view_type+'()')       # get BTPATHTUPLE
        tup              = [option_path.parent]+list(BackestPathTuple)  # build tuple
        tup              = [option_name if x==None else x for x in tup] # overwrite None in BackestPathTuple with backtest_name
        filepath         = reduce(lambda x,y:x/y,tup)                   # combine into single Path object
        filepath         = Path(str(filepath) + '.csv')                 # append .csv suffix
        
        # return
        return filepath
        
    def _logging(self):
        print(self.name,':','REFERENCE',':','DATA_DICO SIZE =',len(self._data_dico))
        for k,v in self.reference.items():
            if k != 'df':
                print(self.name,':','REFERENCE',':',k,':',v)
            else:
                print(self.name,':','REFERENCE',':',k,':',len(v))

    @classmethod
    def dev(self) -> BTPATHTUPLE :
        return DATA_CONTAINER._dev
    
    @classmethod
    def daily(self) -> BTPATHTUPLE :
        '''
        purpose
             extraday summary --> macro extraday overview

        dump frequency
            once a day at end of day

        example files
            ./backtest_name/daily/summary_yyyymmdd.csv

        contents of each daily file
            book|pnl|stock volume|future volume|fees
            All|.|.|.|.
            Trading|.|.|.|.
            Quote|.|.|.|.
            Hedge|.|.|.|.

        used to compute summary for entire backtest
            book|return bps|sharpe|daily pnl|daily stock volume|daily future volume|fee bps
            All|.|.|.|.|.|.
            Trading|.|.|.|.|.|.
            Quote|.|.|.|.|.|.
            Hedge|.|.|.|.|.|.
        '''
        return DATA_CONTAINER._daily
    
    @classmethod
    def extraday(self) -> BTPATHTUPLE :
        '''
        purpose
            extraday behavioural analysis --> customised macro extraday overview

        dump frequency
            once a day at end of day
            reference symbol `Symbol` is used to initialize the dump

        example files
            ./backtest_name/extraday/Trading_yyyymmdd.csv
            ./backtest_name/extraday/Quote_yyyymmdd.csv
            ./backtest_name/extraday/Hedge_yyyymmdd.csv

        contents of each daily file
            TimeStamp|Symbol|PnlTotal|PnlJour|PnlVeille|OpenNom|OpenBidNom|OpenAskNom|...
            EOD|VALE3.SA|.|.|.|.|.|.|...
        '''
        return DATA_CONTAINER._extraday
    
    @classmethod
    def intraday(self) -> BTPATHTUPLE :
        '''
        purpose
            intraday behavioural analysis --> customised macro intraday overview

        dump frequency
            n-minutely snapshots throughout the day
            reference symbol `Symbol` is used to initialize the dump

        example files
            ./backtest_name/intraday/Trading_yyyymmdd.csv
            ./backtest_name/intraday/Quote_yyyymmdd.csv
            ./backtest_name/intraday/Hedge_yyyymmdd.csv

        contents of each daily file
            TimeStamp|Symbol|PnlTotal|PnlJour|PnlVeille|OpenNom|OpenBidNom|OpenAskNom|...
            10:00:00|VALE3.SA|.|.|.|.|.|.|...
            10:01:00|VALE3.SA|.|.|.|.|.|.|...
            ...
            16:59:00|VALE3.SA|.|.|.|.|.|.|...
            17:00:00|VALE3.SA|.|.|.|.|.|.|...
        '''
        return DATA_CONTAINER._intraday

    @classmethod
    def factor(self) -> BTPATHTUPLE :
        '''
        purpose
            extraday factor analysis --> customised macro factor analysis

        dump frequency
            once a day at end of day
            dump on all symbols

        example files
            ./backtest_name/extraday/Trading_yyyymmdd.csv
            ./backtest_name/extraday/Quote_yyyymmdd.csv
            ./backtest_name/extraday/Hedge_yyyymmdd.csv

        contents of each daily file
            Symbol|PnlTotal|PnlJour|PnlVeille|ExecNom|MedLongNom|MaxLongLongNom|MedOpenBidNom|MedOpenAskNom|...
            sym_0|
            sym_1|
            ...
            sym_n|
        '''
        return DATA_CONTAINER._factor
    
        '''
    
            self.raw_data=
        self.filtered_data=self.apply_date_filters()
        

    def apply_date_filters(self):
        # check date validity
        
        # remember filter dates applied
        min_date=self.settings['date_from'].value
        max_date=self.settings['date_to'].value

        # apply date filter
        if min_date==None and max_date==None:
            print('no filter')
            self.filtered_data=self.raw_data
        elif min_date!=None and max_date==None:
            print('filter min_date')
            self.filtered_data=self.raw_data[self.raw_data.Date>=min_date]
        elif min_date==None and max_date!=None:
            print('filter max_date')
            self.filtered_data=self.raw_data[self.raw_data.Date<=max_date]
        else:
            print('filter both')
            self.filtered_data=self.raw_data[self.raw_data.Date.between(min_date,max_date)]
            
        # debugging
        print(min_date)
        print(max_date)
        print(self.raw_data.shape)
        print(self.filtered_data.shape)
    '''

In [ ]:
import ipydatetime
import ipywidgets
import datetime


In [ ]:

time_picker = ipydatetime.TimePicker()
time_picker

In [ ]:
# __file__ exists if notebook called with %run
# e.g. only do example if called directly
try:
    __file__
except NameError:
    # example
    a=DATA_CONTAINER()
    display(dir(a))

In [ ]:
'''
help(DATA_CONTAINER.daily)

help(DATA_CONTAINER.extraday)

help(DATA_CONTAINER.intraday)

help(DATA_CONTAINER.factor)
'''
None